In [1]:
import os
import json
import itertools

Given an enzyme file

1. Parse the EC number
2. Get all genes and their sequence for that EC number
3. Encode all sequences as molecular signature
4. Get compound ids for that EC number
5. Get molecular signature for the compounds
6. Cross product sequences and compunds molecular signature

In [2]:
ec2seq_dir_path = "./data/gene_info/"
all_ec2seq_files = [os.path.join(ec2seq_dir_path, f.name) for f in os.scandir(ec2seq_dir_path) if f.name.endswith(".txt")] 

In [3]:
ec2subs_file_path = "./data/primary_substrate_cid_byECid.json"
ec2subs_file = json.load(open(ec2subs_file_path, "r"))

In [4]:
training_datafile = open("./data/training_raw.csv", "w")
training_fastafile = open("./data/training_fasta.fa", "w")

i = 0
i100 = len(all_ec2seq_files)
i10 = i100//10
progress = list(range(i10, i100, i10))
fasta_set = set()

for ec2seqfile in all_ec2seq_files:
    
    if i in progress:
        print("*", end="")
    
    # getting all the sequences mapped to an ec
    seq_file_stream = open(ec2seqfile, "r")
    seq_name = []
    seq = []
    for line in seq_file_stream.readlines():
        if line.startswith(">"):
            parsed_line = line.strip().replace(">", "")
            seq_name.append(parsed_line)
        else:
            parsed_line = line.strip()
            if len(parsed_line) > 50:
                seq.append(parsed_line)
            else:
                print(len(seq))
                _ = seq_name.pop()
            
    
    ec_num = os.path.basename(ec2seqfile).replace("ec:", "").replace(".txt", "")
    subs = ec2subs_file[ec_num]
    enz_info = zip(seq_name, seq)
    
    training_info = list(itertools.product(enz_info, subs))
    
    for info in training_info:
        seq_name = info[0][0]
        seq = info[0][1]
        subs = info [1]
        
        training_datafile.write(f"{seq_name},{seq},{subs}\n")
        if seq_name not in fasta_set:
            fasta_set.add(seq_name)
            training_fastafile.write(f">{seq_name}\n{seq}\n")
    
    i+=1

training_datafile.close()    

**********

In [5]:
training_fastafile.close()

In [3]:
with open("./data/training_fasta.fa", "r") as f:
    for line in f:
        if line.startswith(">"):
            pass
        else:
            parsed_line = line.strip()
            if len(parsed_line) < 50:
                print(parsed_line)